In [2]:
%load_ext autoreload
%autoreload 2
%pylab inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import pickle
import re
import json
from open_spiel.python.examples.ubc_utils import *
from open_spiel.python.examples.ubc_decorators import *
from open_spiel.python.examples.straightforward_agent import StraightforwardAgent
from open_spiel.python import rl_environment


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/home/ubuntu/.pyenv/versions/venv/lib/python3.8/site-packages/IPython/core/magics/pylab.py:162: UserWarning: pylab import has clobbered these variables: ['logging', 'solve', 'random']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [3]:
game_name = 'big_game_2'
game_config = json.load(open(f"{os.environ['CLOCK_AUCTION_CONFIG_DIR']}/{game_name}.json"))
game = smart_load_sequential_game('clock_auction', game_parameters=dict(filename=f'{game_name}.json'))


Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/big_game_2.json
Done config parsing


In [4]:
x = np.array(game.new_initial_state().child(0).child(0).child(10).child(10).child(10).child(10).information_state_tensor())
x[x.nonzero()]

array([  1.        ,   1.        ,   3.        ,   5.        ,
        19.5       ,  24.5       ,  29.25      ,  34.25      ,
        48.75      ,  53.75      ,  58.5       ,  63.5       ,
        78.        ,  83.        ,   2.5       ,  13.44999981,
        15.94999981,  20.17499924,  22.67499924,  33.625     ,
        36.125     ,  40.34999847,  42.84999847,  53.79999924,
        56.29999924, 200.        , 242.        ,  99.82499695,
        66.55000305,  27.5       ,   2.        ,   1.        ,
         4.        ,   2.        ,   1.        , 300.        ,
       120.        ,  80.        ,  30.        ,  75.        ,
        50.        ,  25.        ,   2.        ,   1.        ,
         2.        ,   1.        ,   4.        ,   2.        ,
        82.5       ,  55.        ,  25.        ,   2.        ,
         1.        ,   2.        ,   1.        ,   4.        ,
         2.        ,  90.75      ,  60.5       ,  25.        ])

In [5]:
env = rl_environment.Environment(game)
num_actions = env.action_spec()["num_actions"]
time_step = env.reset()

agents = [TakeSingleActionDecorator(StraightforwardAgent(i, game_config, num_actions), num_actions) for i in range(game.num_players())]

while not time_step.last():
    player_id = time_step.observations["current_player"]
    agent = agents[player_id]
    agent_output = agent.step(time_step)
    print(player_id, agent_output.action)
    action_list = [agent_output.action]
    time_step = env.step(action_list)

for i, agent in enumerate(agents):
    agent.step(time_step)



0 11
1 11
0 10
1 11
0 10
1 11
0 10
1 3
0 8
1 1
